# Project Euler
## Problems 61 - 70

### ************************

## Problem #61

Find the sum of the only ordered set of six cyclic -digit numbers for which each polygonal type: triangle, square, pentagonal, hexagonal, heptagonal, and octagonal, is represented by a different number in the set.

In [14]:
def cyclicalFigurateNumbers():
    fourDigs = generateFourDigitFigurates()
    for val in fourDigs.values():
        if checkTwoDigits(val, fourDigs, []):
            break

In [16]:
def checkTwoDigits(pairs, fourDigs, figList):
    if len(figList) == 6:
        val = figList[-1][0]
        if val in fourDigs and figList[0] in fourDigs[val]:
            print(figList)
            print(cyclicSum(figList))
            return True
        else:
            return False
    shapes = set()
    if len(figList) > 0:
        shapes = set(x[1] for x in figList)
    truthSet = set()
    for pair in pairs:
        two, fg = pair
        if fg in shapes or two not in fourDigs:
            continue
        truthSet.add(checkTwoDigits(fourDigs[two], fourDigs, figList + [pair]))
    if True in truthSet:
        return True
    return False

In [8]:
def generateFourDigitFigurates():
    twoDigs = {}
    n = 1
    while n * (n + 1) // 2 < 10000:
        tr = n * (n+1) // 2
        sq = n**2
        pe = n * (3*n - 1) // 2
        hx = n * (2*n - 1)
        hp = n * (5*n - 3) // 2
        oc = n * (3*n - 2)
        figs = [tr, sq, pe, hx, hp, oc]
        for i, fig in enumerate(figs):
            if 999 < fig and fig < 10000:
                firstTwo = (fig - (fig % 100)) // 100
                lastTwo = fig % 100
                if lastTwo > 9:
                    if firstTwo in twoDigs:
                        twoDigs[firstTwo].append([lastTwo, i+3])
                    else:
                        twoDigs[firstTwo] = [[lastTwo, i+3]]
        n += 1
    return twoDigs

In [12]:
def cyclicSum(pairs):
    total = 0
    twos = [x[0] for x in pairs]
    for i in range(len(twos)):
        total += twos[i]*100 + twos[(i+1) % len(twos)]
    return total

In [17]:
cyclicalFigurateNumbers()

[[28, 6], [82, 5], [56, 3], [25, 4], [12, 7], [81, 8]]
28684


## Problem #62

Find the smallest cube for which exactly five permutations of its digits are cube.

In [38]:
def smallestFivePermCube():
    n = 1
    tupDict = {}
    notFound = True
    while notFound:
        cube = n**3
        digCount = digitCountTup(cube)
        if digCount in tupDict:
            tupDict[digCount].append(cube)
            if len(tupDict[digCount]) == 5:
                print(tupDict[digCount])
                return min(tupDict[digCount])
        else:
            tupDict[digCount] = [cube]
        n += 1
    return

In [18]:
def digitCountTup(n):
    counts = [0 for _ in range(10)]
    while n > 0:
        n, rem = divmod(n, 10)
        counts[rem] += 1
    return tuple(counts)

In [39]:
smallestFivePermCube()

[127035954683, 352045367981, 373559126408, 569310543872, 589323567104]


127035954683

## Problem #63

How many n-digit positive integers exist which are also an nth power?

In [40]:
import math

In [41]:
def allNthDigitPowers():
    n = 1
    k = math.ceil(10**((n-1)/n))
    total = 0
    while k < 10:
        total += 10 - k
        n += 1
        k = math.ceil(10**((n-1)/n))
    return total

In [42]:
allNthDigitPowers()

49

## Problem #64

How many continued fractions for N <= 10,000 have an odd period?

In [25]:
import math

In [38]:
def oddPeriodFractions(cap):
    squares = set(x**2 for x in range(math.ceil(cap**0.5) + 1))
    count = 0
    for n in range(2, cap+1):
        if n in squares:
            continue
        if continuedFractionLength(n) % 2 == 1:
            count += 1
    return count

In [33]:
def continuedFractionLength(n):
    root = n**0.5
    a, b = 1, math.floor(root)
    i = 0
    pairs = {}
    pairs[(a, b)] = i
    while True:
        i += 1
        if n == b**2:
            print(a, b, n)
        k = math.floor((a * root + a * b) / (n - b**2))
        a, b = (n - b**2) // a, -1 * (b - k*((n - b**2) // a))
        if (a, b) in pairs:
            return i - pairs[(a, b)]
        pairs[(a, b)] = i

In [45]:
oddPeriodFractions(10000)

1322

## Problem #65

Find the sum of digits in the numerator of the 100th convergent of the continued fraction for e.

In [19]:
def eDigitSum(n):
    cons = eConvergents(n)
    frac = [cons[-1], 1]
    for i in range(n-2, -1, -1):
        nextVal = cons[i]
        n, d = frac
        frac = simplifyFrac(n * nextVal + d, n)
    return digSum(frac[0])        

In [17]:
def eConvergents(n):
    terms = [2]
    i = 1
    k = 2
    while i <= n:
        vals = [1, k, 1]
        for val in vals:
            terms.append(val)
            i += 1
        k += 2
    while i > n:
        terms.pop()
        i -= 1
    return terms

In [4]:
def digSum(n):
    total = 0
    while n > 0:
        n, rem = divmod(n, 10)
        total += rem
    return total

In [8]:
def simplifyFrac(n, d):
    c = gcd(n, d)
    return [n // c, d // c]

In [9]:
def gcd(a, b):
    if b == 0:
        return a
    return gcd(b, a % b)

In [21]:
eDigitSum(100)

272

## Problem #66

Find the value of D <= 1,000 in minimal solutions of x for which the largest value of x is obtained in x^2 - D*y^2 = 1.

In [24]:
import math

In [130]:
def bigDiophantine(cap):
    squares = set(x**2 for x in range(math.ceil(cap**0.5) + 1))
    maxD, maxX, maxY = 0, 0, 0
    for i in range(2, cap+1):
        if i in squares:
            continue
        val = diophantineConvergent(i)
        if val[0] > maxX:
            maxX, maxY = val
            maxD = i
    print(f'X: {maxX}, Y: {maxY}')
    return maxD

In [127]:
def diophantineConvergent(n):
    D = n
    period = continuedFractionPeriod(D)
    start = len(period) - 1 if len(period) < 3 else len(period) - 2
    frac = [period[start], 1]
    for i in range(start - 1, -1, -1):
        nextVal = period[i]
        n, d = frac
        frac = simplifyFrac(n * nextVal + d, n)
    if frac[0]**2 - D * frac[1]**2 == 1:
        return frac
    frac = [period[start], 1]
    for i in range(start - 1, 0, -1):
        nextVal = period[i]
        n, d = frac
        frac = simplifyFrac(n * nextVal + d, n)
    for i in range(len(period)-1, -1, -1):
        nextVal = period[i]
        n, d = frac
        frac = simplifyFrac(n * nextVal + d, n)
    return frac

In [89]:
def continuedFractionPeriod(n):
    root = n**0.5
    a, b = 1, math.floor(root)
    i = 0
    pairs = {}
    pairs[(a, b)] = i
    period = [b]
    while True:
        i += 1
        if n == b**2:
            print(a, b, n)
        k = math.floor((a * root + a * b) / (n - b**2))
        period.append(k)
        a, b = (n - b**2) // a, -1 * (b - k*((n - b**2) // a))
        if (a, b) in pairs:
            return period
        pairs[(a, b)] = i

In [90]:
def simplifyFrac(n, d):
    c = gcd(n, d)
    return [n // c, d // c]

In [91]:
def gcd(a, b):
    if b == 0:
        return a
    return gcd(b, a % b)

In [131]:
bigDiophantine(1000)

X: 16421658242965910275055840472270471049, Y: 638728478116949861246791167518480580


661

## Problem #67

Find the maximum total from top to bottom in the triangle, a 15K text file containing a triangle with one-hundred rows.

In [132]:
def maxTopBottom(arr):
    for r in range(1, len(arr)):
        arr[r][0] += arr[r-1][0]
        for c in range(1, len(arr[r])-1):
            arr[r][c] += max(arr[r-1][c-1], arr[r-1][c])
        arr[r][-1] += arr[r-1][-1]
    return max(arr[-1])

In [133]:
def getTriangleArray(filename):
    arr = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            row = [int(x) for x in line.split()]
            arr.append(row)
    return arr

In [134]:
maxTopBottom(getTriangleArray('trianglePath_p67.txt'))

7273

## Problem #68

Using the numbers 1 to 10, and depending on arrangements, it is possible to form 16- and 17-digit strings. What is the maximum 16-digit string for a "magic" 5-gon ring?

In [150]:
def fiveGonRing():
    ring = [5,4,3,2,1]
    for i in range(1, fac(5)):
        inner = nthLexicographic(i, ring)
        outer = [6]
        magic = outer[0] + inner[0] + inner[1]
        numDic = {7:0, 8:0, 9:0, 10:0}
        found = True
        for j in range(1, 5):
            val = magic - inner[j] - inner[(j+1)%5]
            if val not in numDic or numDic[val] == 1:
                found = False
                break
            numDic[val] = 1
            outer.append(val)
        if found:
            digits = []
            for outie, innie in zip(outer, inner):
                digits.append(outie)
                digits.append(innie)
                digits.append(magic - outie - innie)
            return ''.join([str(x) for x in digits])

In [136]:
def nthLexicographic(n, digits):
    if n <= 1 or len(digits) == 1:
        return digits
    f = fac(len(digits)-1)
    if n <= f:
        return [digits[0]] + nthLexicographic(n, digits[1:])
    q, rem = divmod(n-1, f)
    if q+1 >= len(digits):
        return [digits[-1]] + nthLexicographic(rem+1, digits[:len(digits)-1])
    return [digits[q]] + nthLexicographic(rem+1, digits[:q] + digits[q+1:])

In [140]:
def fac(n):
    prod = 1
    for i in range(2, n+1):
        prod *= i
    return prod

In [151]:
fiveGonRing()

'6531031914842725'

## Problem #69

Find the value of n <= 1,000,000 for which n / phi(n) is a maximum.

In [168]:
def maxTotientRatio(cap):
    primes = generatePrimesTo(100)
    prod = 1
    idx = 0
    while prod <= cap:
        prod *= primes[idx]
        idx += 1
    return prod // primes[idx-1]

In [152]:
def generatePrimesTo(n):
    primes = [2,3,5,7]
    i = 11
    while i < n:
        for j in [0, 2, 6, 8]:
            if isPrime(i+j, primes):
                primes.append(i+j)
        i += 10
    while primes[-1] > n:
        primes.pop()
    return primes

In [153]:
def isPrime(n, primes):
    if n < 11:
        return False if n not in [2,3,5,7] else True
    i = 0
    root = math.floor(n**0.5)
    while i < len(primes) and primes[i] <= root:
        if n % primes[i] == 0:
            return False
        i += 1
    return True

In [169]:
maxTotientRatio(1000000)

510510

## Problem #70

Find the value of n, where 1 < n < 10^7 and phi(n) is a permutation of n and n / phi(n) is minimized. 

In [176]:
import math

In [189]:
def minTotientPerm(cap):
    primes = generatePrimesTo(math.ceil(cap**0.5))
    minRatio = float("inf")
    minN = 0
    for n in range(2, cap):
        totient = phi(n, primes)
        val = n / totient
        if val < minRatio and isPerm(n, totient):
            minRatio = val
            minN = n
    return minN

In [184]:
def phi(n, primes):
    totient = n
    root = n**0.5
    for p in primes:
        if p > root:
            break
        if n % p == 0:
            totient //= p
            while n % p == 0:
                n //= p
            totient *= p - 1
            root = n**0.5
    return totient if n < 2 else totient * (n - 1) // n

In [170]:
def generatePrimesTo(n):
    primes = [2,3,5,7]
    i = 11
    while i < n:
        for j in [0, 2, 6, 8]:
            if isPrime(i+j, primes):
                primes.append(i+j)
        i += 10
    while primes[-1] > n:
        primes.pop()
    return primes

In [171]:
def isPrime(n, primes):
    if n < 11:
        return False if n not in [2,3,5,7] else True
    i = 0
    root = math.floor(n**0.5)
    while i < len(primes) and primes[i] <= root:
        if n % primes[i] == 0:
            return False
        i += 1
    return True

In [187]:
def isPerm(a, b):
    return digCount(a) == digCount(b)

In [186]:
def digCount(n):
    digs = [0 for _ in range(10)]
    while n > 0:
        n, rem = divmod(n, 10)
        digs[rem] += 1
    return digs

In [193]:
minTotientPerm(10**7)

8319823